In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import embedders

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm.notebook import tqdm

# Filter out warnings raised when sampling Wishart distribution in Gaussian mixtures
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

if device != torch.device("cuda"):
    sample_device = torch.device("cpu")
else:
    sample_device = device

print(f"Device: {device}, Sample Device: {sample_device}")

Device: cuda, Sample Device: cuda


# Shared hyperparameters

In [4]:
N_FEATURES = "d_choose_2"
MAX_DEPTH = 3
MODELS = ["product_dt", "product_rf", "tangent_mlp", "ambient_mlp", "tangent_gnn", "ambient_gnn"]
N_SAMPLES = 10

# Single curvature

In [5]:
CURVATURES = [-4, -2, -1, -0.5, -0.25, 0, 0.25, 0.5, 1, 2, 4]
DIM = 2
N_POINTS = 1_000
N_CLASSES = 8
N_CLUSTERS = 32
COV_SCALE_MEANS = 1.0
COV_SCALE_POINTS = 1.0

RESAMPLE_SCALES = False

# SCORE = "f1-micro" if TASK == "classification" else "rmse"

for TASK in ["classification", "regression"]:
    results = []
    SCORE = ["f1-micro", "accuracy"] if TASK == "classification" else ["rmse"]
    my_tqdm = tqdm(total=len(CURVATURES) * N_SAMPLES)
    for i, K in enumerate(CURVATURES):
        for seed in range(N_SAMPLES):
            # Ensure unique seed per trial
            seed = seed + N_SAMPLES * i
            pm = embedders.manifolds.ProductManifold(signature=[(K, DIM)]).to(sample_device)

            # Get X, y
            X, y = embedders.gaussian_mixture.gaussian_mixture(
                pm=pm,
                seed=seed,
                num_points=N_POINTS,
                num_classes=N_CLASSES,
                num_clusters=N_CLUSTERS,
                cov_scale_means=COV_SCALE_MEANS / DIM,
                cov_scale_points=COV_SCALE_POINTS / DIM,
                task=TASK,
            )
            X = X.to(device)
            y = y.to(device)
            pm = pm.to(device)

            model_results = embedders.benchmarks.benchmark(
                X, y, pm, max_depth=MAX_DEPTH, task=TASK, score=SCORE, seed=seed, n_features=N_FEATURES, device=device, models=MODELS,
            )

            # Create a flat dictionary for this run
            run_results = {"curvature": K, "seed": seed}

            # Flatten the nested model results
            for model, metrics in model_results.items():
                for metric, value in metrics.items():
                    run_results[f"{model}_{metric}"] = value

            results.append(run_results)
            my_tqdm.update(1)

    # Convert to DataFrame
    results = pd.DataFrame(results)

    results.to_csv(f"embedders/data/results/{TASK}_nn_single_curvature.tsv", sep="\t", index=False)

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

In [6]:
# Multiple curvatures

# Signatures - using non-Gu approach for now
SIGNATURES = [
    [(-1, 2), (-1, 2)],  # HH
    [(-1, 2), (0, 2)],  # HE
    [(-1, 2), (1, 2)],  # HS
    [(1, 2), (1, 2)],  # SS
    [(1, 2), (0, 2)],  # SE
    [(-1, 4)],  # H
    [(0, 4)],  # E
    [(1, 4)],  # S
]

SIGNATURES_STR = ["HH", "HE", "HS", "SS", "SE", "H", "E", "S"]

DIM = 4
N_POINTS = 1_000
N_CLASSES = 8
N_CLUSTERS = 32
MAX_DEPTH = None
COV_SCALE_MEANS = 1.0
COV_SCALE_POINTS = 1.0

for TASK in ["classification", "regression"]:
    results = []
    SCORE = ["f1-micro", "accuracy"] if TASK == "classification" else ["rmse"]

    my_tqdm = tqdm(total=len(SIGNATURES) * N_SAMPLES)
    for i, (sig, sigstr) in enumerate(zip(SIGNATURES, SIGNATURES_STR)):
        for seed in range(N_SAMPLES):
            # Ensure unique seed per trial
            seed = seed + N_SAMPLES * i
            pm = embedders.manifolds.ProductManifold(signature=sig, device=sample_device)

            # Get X, y
            X, y = embedders.gaussian_mixture.gaussian_mixture(
                pm=pm,
                seed=seed,
                num_points=N_POINTS,
                num_classes=N_CLASSES,
                num_clusters=N_CLUSTERS,
                cov_scale_means=COV_SCALE_MEANS / DIM,
                cov_scale_points=COV_SCALE_POINTS / DIM,
                task=TASK,
            )
            X = X.to(device)
            y = y.to(device)
            pm = pm.to(device)

            if RESAMPLE_SCALES:
                scale = 0.5 - np.random.rand() * 20
                pm.P[0].scale = torch.exp(torch.tensor(scale)).item()
                pm.P[0].manifold._log_scale = torch.nn.Parameter(torch.tensor(scale))

            # Benchmarks are now handled by the benchmark function
            model_results = embedders.benchmarks.benchmark(
                X, y, pm, max_depth=MAX_DEPTH, task=TASK, score=SCORE, seed=seed, n_features=N_FEATURES, device=device
            )
            
            # Create a flat dictionary for this run
            run_results = {"signature": sigstr, "seed": seed}

            # Flatten the nested model results
            for model, metrics in model_results.items():
                for metric, value in metrics.items():
                    run_results[f"{model}_{metric}"] = value

            results.append(run_results)
            my_tqdm.update(1)


    results = pd.DataFrame(results)

    results.to_csv(f"embedders/data/results/{TASK}_nn_multiple_curvatures.tsv", sep="\t", index=False)

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

# Graph embeddings - known matrix

In [9]:
GRAPHS = [
    ("citeseer", "HS", [(-1, 2), (1, 2)], "classification"),
    ("cora", "H", [(-1, 4)], "classification"),
    ("polblogs", "SS", [(1, 2), (1, 2)], "classification"),
    ("cs_phds", "H", [(-1, 4)], "regression"),
]

results = []
my_tqdm = tqdm(total=len(GRAPHS) * N_SAMPLES)
for embedding, sigstr, sig, task in GRAPHS:
    _, y, adj = embedders.dataloaders.load(embedding)
    adj = adj.to(device).float().detach()
    pm = embedders.manifolds.ProductManifold(signature=sig, device=device)

    for i in range(N_SAMPLES):
        X = torch.tensor(np.load(f"embedders/data/graphs/embeddings/{embedding}/{sigstr}_{i}.npy"), device=device)
        score = ["f1-micro", "accuracy"] if task == "classification" else ["rmse"]

        model_results = embedders.benchmarks.benchmark(
            X,
            y,
            pm,
            max_depth=MAX_DEPTH,
            n_features=N_FEATURES,
            seed=seed,
            device=device,
            adj=adj,
            models=MODELS,
            task=task,
            score=score,
        )

        # Create a flat dictionary for this run
        run_results = {"embedding": embedding, "seed": i}

        # Flatten the nested model results
        for model, metrics in model_results.items():
            for metric, value in metrics.items():
                run_results[f"{model}_{metric}"] = value

        results.append(run_results)
        my_tqdm.update(1)

results = pd.DataFrame(results)
results.to_csv(f"embedders/data/results/{TASK}_nn_graph.tsv", sep="\t", index=False)

  0%|          | 0/40 [00:00<?, ?it/s]

Top CC has 2110 nodes; original graph has 3264 nodes.
Top CC has 2485 nodes; original graph has 2708 nodes.
Top CC has 1222 nodes; original graph has 1490 nodes.
Top CC has 1025 nodes; original graph has 1025 nodes.


In [11]:
# Summarize data

def ci95_agg(vals):
    return f"{np.mean(vals):.3f} ± {1.96 * np.std(vals):.3f}"

results.groupby("embedding").agg(ci95_agg)

,seed,product_dt_f1-micro,product_dt_accuracy,product_dt_time,product_rf_f1-micro,product_rf_accuracy,product_rf_time,tangent_mlp_f1-micro,tangent_mlp_accuracy,tangent_mlp_time,...,ambient_gnn_accuracy,ambient_gnn_time,product_dt_rmse,product_rf_rmse,tangent_mlp_rmse,ambient_mlp_rmse,tangent_gnn_adj_rmse,ambient_gnn_adj_rmse,tangent_gnn_rmse,ambient_gnn_rmse
embedding,,,,,,,,,,,,,,,,,,,,,
citeseer,4.500 ± 5.630,0.226 ± 0.036,0.226 ± 0.036,1.261 ± 0.143,0.239 ± 0.039,0.239 ± 0.039,11.388 ± 0.392,0.234 ± 0.034,0.234 ± 0.034,0.125 ± 0.007,...,0.248 ± 0.026,2.100 ± 0.007,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan
cora,4.500 ± 5.630,0.186 ± 0.029,0.186 ± 0.029,1.518 ± 0.049,0.216 ± 0.039,0.216 ± 0.039,13.297 ± 0.450,0.287 ± 0.031,0.287 ± 0.031,0.125 ± 0.004,...,0.275 ± 0.081,2.997 ± 0.010,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan
cs_phds,4.500 ± 5.630,nan ± nan,nan ± nan,0.943 ± 0.043,nan ± nan,nan ± nan,8.081 ± 0.288,nan ± nan,nan ± nan,0.133 ± 0.003,...,nan ± nan,0.425 ± 0.005,17.855 ± 5.962,13.162 ± 3.191,1978.597 ± 2.238,1978.597 ± 2.238,1978.597 ± 2.238,1978.597 ± 2.238,1978.597 ± 2.238,1978.597 ± 2.238
polblogs,4.500 ± 5.630,0.890 ± 0.189,0.890 ± 0.189,0.098 ± 0.008,0.930 ± 0.021,0.930 ± 0.021,1.129 ± 0.300,0.854 ± 0.245,0.854 ± 0.245,0.128 ± 0.012,...,0.492 ± 0.047,0.495 ± 0.012,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan,nan ± nan
